# Evaluation of models predicting the electricity prices

Run [SARIMA](SARIMA.ipynb), [SARIMAX](SARIMAX.ipynb), and [LSTM](LSTMmodel.ipynb) models to predict the electricity prices and save forecasts as numpy arrays before running this notebook.

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from modelling import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the data

In [23]:
# path to numpy files
sarima = 'output/forecast/forecast_sarima.npy'
sarimax = 'output/forecast/forecast_sarimax.npy'
lstm = 'output/forecast/forecast_LSTM.npy'
data = '../../data/fulldata.csv'

# load numpy files
sarima_forecast = np.load(sarima)
sarimax_forecast = np.load(sarimax)
lstm_forecast = np.load(lstm)

# load test data and create naive forecast
data = pd.read_csv(data)
data = data[['SpotPrice', 'from']]
data['from'] = pd.to_datetime(data['from'])
data = data.set_index('from')
data.sort_index(inplace=True)

# naive forecast
naive = data.shift(24)['SpotPrice'].values

# drop old data
naive = naive[data.index >= '2024-08-01']
data = data[data.index >= '2024-08-01']

# create np series
actuals = data['SpotPrice'].values

# create timestamps for plotting
timestamps = data.index.values
timestamps = pd.to_datetime(timestamps)

In [24]:
_, sarima_mse_p = diebold_mariano_test(actuals, sarima_forecast, naive, loss_function='mse', h=24)
_, sarima_mae_p = diebold_mariano_test(actuals, sarima_forecast, naive, loss_function='mae', h=24)
_, sarimax_mse_p = diebold_mariano_test(actuals, sarimax_forecast, naive, loss_function='mse', h=24)
_, sarimax_mae_p = diebold_mariano_test(actuals, sarimax_forecast, naive, loss_function='mae', h=24)
_, lstm_mse_p = diebold_mariano_test(actuals, lstm_forecast, naive, loss_function='mse', h=24)
_, lstm_mae_p = diebold_mariano_test(actuals, lstm_forecast, naive, loss_function='mae', h=24)

models = ["Naive", "SARIMA", "SARIMAX", "LSTM"]
metrics = {
    "RMSE": [
        root_mean_squared_error(actuals, naive),
        (root_mean_squared_error(actuals, sarima_forecast), sarima_mse_p),
        (root_mean_squared_error(actuals, sarimax_forecast), sarimax_mse_p),
        (root_mean_squared_error(actuals, lstm_forecast), lstm_mse_p)
    ],
    "MAE": [
        mean_absolute_error(actuals, naive),
        (mean_absolute_error(actuals, sarima_forecast), sarima_mae_p),
        (mean_absolute_error(actuals, sarimax_forecast), sarimax_mae_p),
        (mean_absolute_error(actuals, lstm_forecast), lstm_mae_p)
    ]
}

table = latex_table(models, metrics)

with open('output/results.tex', 'w') as f:
    f.write(table)

In [25]:
# Ensure timestamps are datetime
timestamps = pd.to_datetime(timestamps)

# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(28, 16))

# Define models and their labels/colors
models = [
    ("Naive Forecast", naive, '#ff7f0e'),
    ("SARIMA Forecast", sarima_forecast, '#e377c2'),
    ("LSTM Forecast", lstm_forecast, '#d62728'),
    ("SARIMAX Forecast", sarimax_forecast, '#bcbd22'),
]

# Plot each model on its own subplot
for ax, (label, model, color) in zip(axs.flat, models):
    ax.plot(timestamps, actuals, label='Actual Prices', color='#1f77b4', alpha=0.8, linewidth=3)
    ax.plot(timestamps, model, label=label, color = color, alpha=0.7, linewidth=3)
    ax.set_ylabel('Spot Price (DKK per MWh)', fontsize=26)
    ax.tick_params(axis='both', which='major', labelsize=24)
    ax.legend(fontsize=26)
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d, %y"))
    ax.tick_params(axis='x', rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()

# Save and show the plot
plt.savefig('output/predicted_v_actuals_full.png')
print('Plot saved to output/predicted_v_actuals_full.png')
# plt.show()
plt.close()


Plot saved to output/predicted_v_actuals_full.png


In [26]:
plot_forecasts(
    timestamps=timestamps,
    actuals=actuals,
    naive=naive,
    sarima_forecast=sarima_forecast,
    sarimax_forecast=sarimax_forecast,
    lstm_forecast=lstm_forecast,
    start_datetime="2024-08-01 00:00",
    end_datetime="2024-08-8 00:00",
    output_path="output/1stweek_forecasts.png",
    display_plot=False
)


Forecast plot saved to output/1stweek_forecasts.png


In [27]:
plot_forecasts(
    timestamps=timestamps,
    actuals=actuals,
    naive=naive,
    sarima_forecast=sarima_forecast,
    sarimax_forecast=sarimax_forecast,
    lstm_forecast=lstm_forecast,
    start_datetime="2024-11-03 00:00",
    end_datetime="2024-11-10 00:00",
    output_path="output/difficult_forecasts.png",
    display_plot=False
)

Forecast plot saved to output/difficult_forecasts.png


In [28]:
plot_forecast_scatter(
    actuals=actuals,
    naive=naive,
    sarima_forecast=sarima_forecast,
    sarimax_forecast=sarimax_forecast,
    lstm_forecast=lstm_forecast,
    output_path="output/forecast_scatter.png",
    display_plot=False
)


Scatter plot saved to output/forecast_scatter.png


In [33]:
import numpy as np

def calculate_daily_metrics(actuals, forecasts, daily_horizon=24):
    """
    Calculates daily RMSE and MAE for a sequence of daily forecasts.
    
    Args:
        actuals (numpy.ndarray): Array of actual values.
        forecasts (numpy.ndarray): Array of forecasted values.
        daily_horizon (int): Number of time steps in each daily forecast (default is 24 for hourly forecasts).
    
    Returns:
        tuple: Two numpy arrays containing daily RMSE and daily MAE values.
    """
    # Ensure actuals and forecasts are the same length
    assert len(actuals) == len(forecasts), "Actuals and forecasts must have the same length."
    
    # Number of daily forecasts
    num_days = len(actuals) // daily_horizon
    
    # Reshape into daily blocks
    actuals_daily = actuals[:num_days * daily_horizon].reshape(num_days, daily_horizon)
    forecasts_daily = forecasts[:num_days * daily_horizon].reshape(num_days, daily_horizon)
    
    # Calculate RMSE and MAE for each day
    daily_rmse = np.sqrt(np.mean((actuals_daily - forecasts_daily) ** 2, axis=1))
    daily_mae = np.mean(np.abs(actuals_daily - forecasts_daily), axis=1)

    return daily_rmse, daily_mae



In [42]:
models = ['naive','sarima', 'sarimax', 'lstm']
metrics = ['rmse', 'mae']
forecasts = {'naive': naive,'sarima': sarima_forecast, 'sarimax': sarimax_forecast, 'lstm': lstm_forecast}

daily_metrics = {}
stats = {}

for model in models:
    for metric in metrics:
        daily_metrics[f'{model}_{metric}'] = calculate_daily_metrics(actuals, forecasts[model])[0 if metric == 'rmse' else 1]
        metric_values = daily_metrics[f'{model}_{metric}']
        stats[f'{model}_{metric}_mean'] = np.mean(metric_values)
        stats[f'{model}_{metric}_std'] = np.std(metric_values)
        stats[f'{model}_{metric}_min'] = np.min(metric_values)
        stats[f'{model}_{metric}_max'] = np.max(metric_values)
        # Print results
        print(f"\n{model.upper()} {metric.upper()} Statistics:")
        print(f"Mean: {stats[f'{model}_{metric}_mean']:.4f}")
        print(f"Std: {stats[f'{model}_{metric}_std']:.4f}")
        print(f"Min: {stats[f'{model}_{metric}_min']:.4f}")
        print(f"Max: {stats[f'{model}_{metric}_max']:.4f}")


NAIVE RMSE Statistics:
Mean: 316.7349
Std: 158.0895
Min: 28.2419
Max: 763.8354

NAIVE MAE Statistics:
Mean: 253.9520
Std: 142.1265
Min: 24.8296
Max: 720.0171

SARIMA RMSE Statistics:
Mean: 285.4829
Std: 141.1468
Min: 57.3252
Max: 771.2680

SARIMA MAE Statistics:
Mean: 222.4437
Std: 112.7717
Min: 47.7890
Max: 564.8237

SARIMAX RMSE Statistics:
Mean: 282.6916
Std: 135.8206
Min: 59.3511
Max: 750.7002

SARIMAX MAE Statistics:
Mean: 224.6852
Std: 114.7869
Min: 42.9293
Max: 586.3808

LSTM RMSE Statistics:
Mean: 297.2630
Std: 128.9752
Min: 102.2078
Max: 850.2742

LSTM MAE Statistics:
Mean: 249.2722
Std: 106.8776
Min: 84.1831
Max: 599.6422
